In [1]:
"""
Run once ever

Downloads and extracts the dataset
"""


# import urllib.request
# import tarfile
# import os

# mp3file = urllib.request.urlopen("https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz")
# with open('tmp','wb') as output:
#   output.write(mp3file.read())

# file = tarfile.open("tmp")
# file.extractall(".")
# file.close()
# os.remove("tmp")
# os.remove("cifar-100-python/file.txt~")

'\nRun once ever\n\nDownloads and extracts the dataset\n'

In [2]:

import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline
import cv2
from collections import Counter
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.model_selection import GridSearchCV
import threading
import time
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
#function to read files present in the Python version of the dataset
def unpickle(file):
    with open(file, 'rb') as fo:
        myDict = pickle.load(fo, encoding='latin1')
    return myDict

In [4]:
trainData = unpickle('./cifar-100-python/train')
#type of items in each file
for item in trainData:
    print(item, type(trainData[item]))

filenames <class 'list'>
batch_label <class 'str'>
fine_labels <class 'list'>
coarse_labels <class 'list'>
data <class 'numpy.ndarray'>


In [5]:
testData = unpickle('./cifar-100-python//test')
metaData = unpickle('./cifar-100-python//meta')
#metaData
print("Fine labels:", metaData['fine_label_names'], "\n")
print("Coarse labels:", metaData['coarse_label_names'])

Fine labels: ['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table', 'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 'tulip', 'turtle', 'wardrobe', 'whale', '

In [6]:
#storing fine labels along with its number code in a dataframe
subCategory = pd.DataFrame(metaData['fine_label_names'], columns=['SubClass'])
print(subCategory)

         SubClass
0           apple
1   aquarium_fish
2            baby
3            bear
4          beaver
..            ...
95          whale
96    willow_tree
97           wolf
98          woman
99           worm

[100 rows x 1 columns]


In [7]:
# I'm pretty sure this is set up so that dimension 0 is the category
X_train = trainData['data']
X_train = X_train.reshape(len(X_train),3,32,32).transpose(0,2,3,1)

In [8]:
X_test = testData['data']
X_test = X_test.reshape(len(X_test),3,32,32).transpose(0,2,3,1)
#X_test

In [9]:
winSize = (32,32)
blockSize = (8,8)
blockStride = (4,4)
cellSize = (4,4)
nbins = 9


df = []

**Create Database of HOGs** 
(Pandas dataframe)

In [10]:
# this is essentially where we fit the model
DF = []
def getHogs(winSize, blockSize, blockStride, cellSize, nbins):
    global DF
    global df
    df = []
    for i in range(len(X_train)):
    # for i in range(5000):
        image = cv2.cvtColor(X_train[i], cv2.COLOR_BGR2GRAY)
        hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)
        df.append(hog.compute(image))
    # DF = pd.DataFrame(df)
    DF = np.array(df)


In [11]:
getHogs(winSize, blockSize, blockStride, cellSize, nbins)
DF[0]

array([0.03279916, 0.0153011 , 0.04578669, ..., 0.02589341, 0.15030332,
       0.21390088], dtype=float32)

****continue work checkpoint: standardize test data and proceed****

In [12]:
def predict(img, k, winSize, blockSize, blockStride, cellSize, nbins, DF):
    
    #Extract sample's features
    sample = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)
    _hog = hog.compute(sample)
    
    #Get Nearest Neighbors
    distances = []
    for i in range(len(DF)):
        distances.append(np.linalg.norm(_hog - DF[i]))         # 1- Calculate and store each datapoint's Euclidean distance
    # distances = np.linalg.norm((np.expand_dims(_hog, axis=0) - DF))
    
    kNNsID = np.argsort(distances)[:k]                              # 2- Sort distances' IDs in an ascending order
    
    kNNsCats = []
    for i in kNNsID:
        kNNsCats.append(subCategory.iloc[trainData['fine_labels'][i]][0].capitalize())      # 3- Get categories of each of the NNs
    
    CommonCat = Counter(kNNsCats).most_common(1)                    # 4- Count and get Most common category
    
    return CommonCat

In [13]:
n = 25
getHogs(winSize, blockSize, blockStride, cellSize, nbins)
p = predict(X_test[n], 20, winSize, blockSize, blockStride, cellSize, nbins, DF)

/tmp/ipykernel_2348194/1813209976.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  kNNsCats.append(subCategory.iloc[trainData['fine_labels'][i]][0].capitalize())      # 3- Get categories of each of the NNs


In [14]:
def getAccuracy(k, x_test, testdata, winSize, blockSize, blockStride, cellSize, nbins, X_train):
    df = []
    for i in range(len(X_train)):
    # for i in range(5000):
        image = cv2.cvtColor(X_train[i], cv2.COLOR_BGR2GRAY)
        hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)
        df.append(hog.compute(image))
    # DF = pd.DataFrame(df)
    # DF = np.array(df)
    df = np.array(df)
    RightCount = 0
    preds = []
    for i in range(1000):
        pred = predict(x_test[i], k, winSize, blockSize, blockStride, cellSize, nbins, df)
        if pred[0][0] == subCategory.iloc[testdata['fine_labels'][i]][0].capitalize():
            RightCount += 1
        preds.append(pred)
        # print("Done: {}, Accuracy: {}, Time: {}".format(i,(RightCount/(i+1))*100, time.time() % 10000), flush=True)
    # it would be RightCount / 1000 * 100, so I just made it RightCount / 10
    return RightCount / 10

In [15]:

# param_grid = {
#     # "nbins": [5, 7, 9, 11, 13],
#     "nbins": [13],
#     # "blockSize": [(2, 2), (4,4), (8,8)],
#     "blockSize": [(8,8)],
#     "blockStride": [(1,1), (2,2), (4,4), (8,8)],
#     "cellSize": [(1,1), (2,2), (4,4), (8,8)],
#     "k": [4, 7, 10, 14, 18, 25],
# }

# param_grid = {
#     "nbins": [10, 12],
#     # "blockSize": [(2, 2), (4,4), (8,8)],
#     "blockSize": [(8,8)],
#     "blockStride": [(1,1), (4,4)],
#     "cellSize": [(2,2)],
#     "k": [18, 20, 22, 25, 27, 30, 50],
# }

# list of configs that performed well with just 5k samples, but now testing with all of the samples
param_grid = {
    "nbins": [9, 11, 12],
    "blockSize": [(8,8)],
    "blockStride": [(4,4)],
    "cellSize": [(2,2)],
    "k": [10, 14, 18, 22, 25, 27],
    # "k": [10],
}

accuracies = []

def addAccuacy(k, X_test, testData, winSize, bs, bls, c, n, DF):
    # global accuracies
    # print(f"Starting {k}")
    # accuracies.append(getAccuracy(k, X_test, testData, winSize, bs, bls, c, n))
    accuracy = getAccuracy(k, X_test, testData, winSize, bs, bls, c, n, DF)
    accuracies.append(accuracy)
    print(f"testing accuracy {n} | {bs} | {bls} | {c} | {k} | {time.time()}")
    print(f"Accuracy: {accuracy}")

threads = []

for n in param_grid["nbins"]:
    for bs in param_grid["blockSize"]:
        for bls in param_grid["blockStride"]:
            for c in param_grid["cellSize"]:
                # check the two asserts that OpenCV has, and just move on if this configuration won't work
                if not (bs[0] % c[0] == 0 and bs[0] % c[0] == 0):
                    continue
                if not ((winSize[0] - bs[0]) % bls[0] == 0 and (winSize[0] - bs[0]) % bls[0] == 0):
                    continue
                # getHogs(winSize, bs, bls, c, n)
                # print(f"got hogs at {round(time.time())}")
                # print(DF.shape)
                threads = []
                for k in param_grid["k"]:
                    # print(f"testing accuracy {n} | {bs} | {bls} | {c} | {k}")
                    # accuracies.append(getAccuracy(k, X_test, testData, winSize, bs, bls, c, n))
                    threads.append(threading.Thread(target=addAccuacy, args=(k, X_test, testData, winSize, bs, bls, c, n, X_train)))
                    threads[len(threads)-1].start()
                for thread in threads:
                    thread.join()
                    # print(f"Accuracy: {accuracies[len(accuracies) - 1]}")


/tmp/ipykernel_2348194/1813209976.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  kNNsCats.append(subCategory.iloc[trainData['fine_labels'][i]][0].capitalize())      # 3- Get categories of each of the NNs
/tmp/ipykernel_2348194/2059484418.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if pred[0][0] == subCategory.iloc[testdata['fine_labels'][i]][0].capitalize():


testing accuracy 9 | (8, 8) | (4, 4) | (2, 2) | 25 | 1745464386.3993187
Accuracy: 21.2
testing accuracy 9 | (8, 8) | (4, 4) | (2, 2) | 22 | 1745464398.143203
Accuracy: 20.8
testing accuracy 9 | (8, 8) | (4, 4) | (2, 2) | 14 | 1745464403.4008915
Accuracy: 22.7
testing accuracy 9 | (8, 8) | (4, 4) | (2, 2) | 10 | 1745464405.815524
Accuracy: 23.6
testing accuracy 9 | (8, 8) | (4, 4) | (2, 2) | 27 | 1745464407.281623
Accuracy: 21.0
testing accuracy 9 | (8, 8) | (4, 4) | (2, 2) | 18 | 1745464407.7817035
Accuracy: 21.2
testing accuracy 11 | (8, 8) | (4, 4) | (2, 2) | 27 | 1745466980.803638
Accuracy: 20.7
testing accuracy 11 | (8, 8) | (4, 4) | (2, 2) | 10 | 1745467007.8588314
Accuracy: 24.0
testing accuracy 11 | (8, 8) | (4, 4) | (2, 2) | 22 | 1745467027.4064965
Accuracy: 21.3
testing accuracy 11 | (8, 8) | (4, 4) | (2, 2) | 18 | 1745467034.359706
Accuracy: 22.3
testing accuracy 11 | (8, 8) | (4, 4) | (2, 2) | 25 | 1745467037.7091107
Accuracy: 21.2
testing accuracy 11 | (8, 8) | (4, 4) | (2,

In [16]:
# print(accuracies)
print(max(accuracies))

24.6


In [17]:
TruePreds = []

for i in range(1000):
    TruePreds.append(subCategory.iloc[testData['fine_labels'][i]][0].capitalize())

/tmp/ipykernel_2348194/3796635586.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  TruePreds.append(subCategory.iloc[testData['fine_labels'][i]][0].capitalize())


In [18]:
# Predicted = []
# for i in range(len(Preds14)):
#     Predicted.append(Preds14[i][0][0])

In [19]:
# cm = confusion_matrix(TruePreds, Predicted)

In [20]:
# x = np.array(TruePreds)
# len(np.unique(x))

In [21]:
# cm_df = pd.DataFrame(cm,
#                      index = np.unique(x), 
#                      columns = np.unique(x))

In [22]:
# plt.figure(figsize=(16,16))
# sns.heatmap(cm_df, annot=True)
# plt.title('Confusion Matrix')
# plt.ylabel('Actal Values')
# plt.xlabel('Predicted Values')
# plt.show()